In [119]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from flask import Flask, render_template
import pymongo
import requests
from selenium import webdriver



# NASA Mars News


Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
Assign the text to variables that you can reference later.

In [65]:
news_info={}
news_p = []

nasa_url = "https://mars.nasa.gov/news/"      
base_url = "https://mars.nasa.gov/"
response_1 = requests.get(nasa_url)
nasa_soup = bs(response_1.text,'html.parser')

In [66]:
soup_div = nasa_soup.find(class_="slide")                                    
soup_news = soup_div.find_all('a')                                         
news_title = soup_news[1].get_text().strip()

In [67]:
soup_div = nasa_soup.find(class_="slide")                                   
soup_news = soup_div.find_all('a')                                           
news_title = soup_news[1].get_text().strip()    

In [68]:
soup_p = soup_div.find_all('a', href=True)                                  
soup_p_url = soup_p[0]['href']                                             
paragraph_url = base_url + soup_p_url                                        
response_2 = requests.get(paragraph_url)                                         
para_soup = bs(response_2.text, "html.parser")                               
ww_paragraphs = para_soup.find(class_='wysiwyg_content')                    
paragraphs = ww_paragraphs.find_all('p')  

In [69]:
for paragraph in paragraphs:                                                 
    clean_paragraph = paragraph.get_text().strip()                           
    news_p.append(clean_paragraph)   

In [70]:
news_info["news_title"] = news_title 
news_info["news_p"]= news_p
news_info   



{'news_title': 'Why This Martian Full Moon Looks Like Candy',
 'news_p': ["For the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera, which has been studying the Martian moon since September of 2017. Looking like a rainbow-colored jawbreaker, these latest observations could help scientists understand what materials make up Phobos, the larger of Mars' two moons.",
  "Odyssey is NASA's longest-lived Mars mission. Its heat-vision camera, the Thermal Emission Imaging System (THEMIS), can detect changes in surface temperature asPhobos circles Mars every seven hours. Different textures and minerals determine how much heat THEMIS detects.",
  '"This new image is a kind of temperature bullseye — warmest in the middle and gradually cooler moving out," said Jeffrey Plaut, Odyssey project scientist at NASA\'s Jet Propulsion Laboratory in Pasadena, Ca

# JPL Mars Space Images - Featured Image


Visit the url for JPL Featured Space Image here.

Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

Make sure to find the image url to the full size .jpg image.

Make sure to save a complete url string for this image.

In [86]:
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

jpl_nasa_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_nasa_url)

In [87]:
# Find and Click "FULL IMAGE" button
full_image_data = browser.find_by_id('full_image')
full_image_data.click()

In [88]:
time.sleep(2)
more_info_data = browser.find_link_by_partial_text('more info')
more_info_data.click()

In [89]:
time.sleep(2)
mars_images_html = browser.html
mars_images_data = bs(mars_images_html, 'html.parser')

In [90]:
try:    
    featured_image_url = mars_images_data.find('figure', class_='lede').a['href']
    print(f"The featured_image_url is: {featured_image_url}") 
    
except AttributeError as e:
    print(e)

The featured_image_url is: /spaceimages/images/largesize/PIA17044_hires.jpg


In [91]:
#full image url
full_image_url = "https://www.jpl.nasa.gov" + featured_image_url
print(f"The full_image_url is: {full_image_url}")


The full_image_url is: https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17044_hires.jpg


# Mars Weather


Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather.

In [92]:
mars_weather_twitter_url = 'https://twitter.com/marswxreport?lang=en)'
browser.visit(mars_weather_twitter_url)
mars_weather_html = browser.html
mars_weather_data = bs(mars_weather_html, 'html.parser')

In [93]:
mars_weather  = mars_weather_data.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"}).\
find('p',class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
print(f"The mars_weather is: {mars_weather}")

The mars_weather is: InSight sol 167 (2019-05-17) low -100.5ºC (-148.9ºF) high -20.4ºC (-4.6ºF)
winds from the SW at 4.7 m/s (10.6 mph) gusting to 13.5 m/s (30.3 mph)
pressure at 7.50 hPapic.twitter.com/0Eqt9nN21o


# Mars Facts


Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

Use Pandas to convert the data to a HTML table string.

In [94]:
mars_facts_url = 'https://space-facts.com/mars/'
mars_facts_table_df = pd.read_html(mars_facts_url)[0]
mars_facts_table_df.columns = ['Description','Value']
mars_facts_table_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [95]:
#Use Pandas to convert the data to a HTML table string
mars_facts_table_df.to_html('MarsFactsTable.html', index=False )

# Mars Hemispheres


Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.

You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.

Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [111]:
usgs_astrogeology_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(usgs_astrogeology_url)

hemisphere_image_urls = []

usgs_astrogeology_html = browser.html
all_hemisphere_data = bs(usgs_astrogeology_html, 'html.parser')

In [112]:
hemisphere_results = all_hemisphere_data.find('div', class_='collapsible results').find_all('div',class_='item')

In [116]:
# Loop through hemisphere results
for each_hemisphere in hemisphere_results:
    hem_title_old = each_hemisphere.find('div', class_='description').find('a', class_='itemLink product-item').h3.text
# Eliminate the word 'Enhanced' from title
    hem_title = ' '.join(hem_title_old.split()[0:-1])
#print(hem_title)
     
    base_hem_url = 'https://astrogeology.usgs.gov'

    each_hem_image_url = base_hem_url + each_hemisphere.find('a',class_='itemLink product-item')['href']
    
    browser.visit(each_hem_image_url)
    time.sleep(2)
    each_hem_img_html = browser.html
    each_hem_data = bs(each_hem_img_html, 'html.parser')
    full_image_url = each_hem_data.find('div',class_='downloads').a['href']
    
    each_hemisphere_image = {
        "title" : hem_title,
        "image_url" : full_image_url
    }
    print(each_hemisphere_image)

    hemisphere_image_urls.append(each_hemisphere_image)

{'title': 'Cerberus Hemisphere', 'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}
{'title': 'Schiaparelli Hemisphere', 'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}
{'title': 'Syrtis Major Hemisphere', 'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}
{'title': 'Valles Marineris Hemisphere', 'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}
